In [1]:

# pip installs

!pip install -q datasets requests torch peft bitsandbytes transformers trl accelerate sentencepiece


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:

import os
import re
import math
from tqdm import tqdm
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed
from datasets import load_dataset, Dataset, DatasetDict
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig

In [3]:
from huggingface_hub import login


In [ ]:
# Log in to HuggingFace

hf_token = ""
login(hf_token)

In [6]:
# Huggingface repository
new_model = "domain_specific_chatbot_v2"
#replace with username
hf_model_repo="MLsheenu/"+new_model

In [30]:
from datasets import load_dataset
import json

dataset_name = 'MLsheenu/domain_specific_chatbot_dataset_v2'
dataset = load_dataset(dataset_name, split="train")

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "mistralai/Mistral-7B-Instruct-v0.3"
device_map = {"": 0}

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_cache = False,
    device_map=device_map
)
model.config.use_cache = False

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
pip install protobuf

In [12]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [13]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
learning_rate = 1.4e-4
max_grad_norm = 0.3
max_steps = 300 #change the number of epochs here
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    num_train_epochs=2,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [14]:
dataset

Dataset({
    features: ['Question', 'Answer'],
    num_rows: 56904
})

In [31]:
def format_dataset(data_point):
    prompt = f"""Answer the Question

    Question: {data_point['Question']}

    Answer: {data_point['Answer']}
     """
    tokens = tokenizer(prompt,
        truncation=True,
        max_length=600,
        padding="max_length",
                      add_special_tokens=False)
    tokens["labels"] = tokens['input_ids'].copy()
    return tokens

In [32]:
print(dataset[0].keys())

dataset = dataset.map(format_dataset)
print(dataset[0].keys())

dict_keys(['Question', 'Answer'])


Map:   0%|          | 0/56904 [00:00<?, ? examples/s]

dict_keys(['Question', 'Answer', 'input_ids', 'attention_mask', 'labels'])


In [33]:
dataset = dataset.remove_columns(['Question','Answer'])

In [34]:
print(dataset[0].keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [35]:
from trl import SFTTrainer

max_seq_length = 600

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,  # Make sure the dataset is properly formatted
    # eval_dataset=valid_dataset,  # You can uncomment this if you have a validation set
    peft_config=peft_config,  # Ensure PEFT configuration is properly set
    dataset_text_field="text",  # Ensure dataset uses the correct 'text' field
    max_seq_length=max_seq_length,  # Ensure this is appropriate for your tokenized sequences
    tokenizer=tokenizer,  # Ensure the tokenizer is correctly set up
    args=training_arguments,  # Ensure training arguments are correctly configured
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. T

In [36]:
trainer.train()

Step,Training Loss
500,1.285900
1000,1.214300
1500,1.193300
2000,1.178100
2500,1.185400
3000,1.188000
3500,1.166300
4000,1.153700
4500,1.156000
5000,1.146900


TrainOutput(global_step=7112, training_loss=1.1767242504587592, metrics={'train_runtime': 46702.4109, 'train_samples_per_second': 2.437, 'train_steps_per_second': 0.152, 'total_flos': 2.92534740320256e+18, 'train_loss': 1.1767242504587592, 'epoch': 1.9997188246871924})

In [38]:
trainer.model.push_to_hub("Domain_based_chatbot_mistral_v2", private=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/MLsheenu/Domain_based_chatbot_mistral_v2/commit/17d86640b6d2cb293a3e5d10b0a779a9a1f0fc55', commit_message='Upload model', commit_description='', oid='17d86640b6d2cb293a3e5d10b0a779a9a1f0fc55', pr_url=None, repo_url=RepoUrl('https://huggingface.co/MLsheenu/Domain_based_chatbot_mistral_v2', endpoint='https://huggingface.co', repo_type='model', repo_id='MLsheenu/Domain_based_chatbot_mistral_v2'), pr_revision=None, pr_num=None)